# Dependências

In [ ]:
# Instala dependências
!pip install -r ../../../requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Leitura dos arquivos

In [ ]:
dados = pd.read_csv('../../../data/dados_cvm.csv')

In [ ]:
dados_tratados = pd.read_csv('../../../data/tratado/dados_cvm_tratados.csv')

# Contextualização
Com a definição do modelo final, utilizando o *Isolation Forest*, é necessário realizar a retestagem das hipóteses analisadas previamente, verificando se os resultados trazidos pelo algoritmo convergem com a análise feita. Entretanto, alguns pontos precisam ser ajustados e introduzidos com antecedência.

Em primeiro lugar, a hipótese 1, referente à relação da quantidade de cedentes com o valor de risco, não fará parte dos novos testes, haja vista a insuficiência de dados no *dataset* enviado pela CVM correspondente às informações dos cedentes dos fundos. Essa base de dados possui muitos registros nulos e incoerentes, impossibilitando a análise ou tornando-a improdutiva.

De forma semelhante, a hipótese 6 também não passará por novos testes, dado que a visão generalista que ela requer sobre os fundos de investimento em direitos creditórios não pode ser obtida pela análise de uma amostra tão pequena como a utilizada, correspondente aos fundos retornados pelo modelo *Isolation Forest* do grupo FIDCAS.

Além disso, a hipótese de novo valor de risco também não será testada novamente, pois já foi descartada pelo grupo pela falta de precisão e grande número de falsos positivos gerados pela alteração da fórmula. Verificou-se também que a fórmula original, apesar de não tornar viável a criação de um modelo preditivo eficiente, possui valor para análise e modelagem, não havendo necessidade de mudança.

Por fim, é importante salientar que os fundos detectados como anomalias foram identificados de acordo com diversas *features* dos dados e em relação ao conjunto de fundos que compartilhava do mesmo tipo de carteira, logo, podem não representar uma amostra adequada para a testagem das hipóteses, mas é válido verificar os resultados dos testes ainda assim.

Hipóteses não testadas:

1 - "FIDCs mais concentrados em poucos cedentes (campos I.a.12 e I.b.12) tendem a ter aumento de VR ao longo dos meses? Qual o nível de concentração de cedentes antecede um aumento do VR? A soma dos percentuais relativos aos cedentes, informados ou não no campo I.a.12, não necessariamente somam 100%, tampouco os do campo I.b.12."

6 - "De forma geral, DCs com e sem riscos e benefícios comportam-se forma diferente ou não, quanto à evolução da inadimplência dos seus créditos e do VR?"

### Identificadores ("ID_Participante") dos fundos detectados pelo modelo *Isolation Forest* para testes

In [ ]:
ids = [198283, 158266, 182033, 179275, 188596, 70133, 152842, 161657, 112864, 71286, 159472, 43438, 76542, 60923, 117007,
       127236, 181942, 59871, 115933, 73687, 170378, 165933, 129742, 120200, 70810, 73687, 68016, 44682, 183463, 209471, 158266]

### Organização dos dados para construção de gráficos

In [ ]:
# preenchimento de dados nulos com zeros e drop de registros com patrimônio líquido zerado (para poder realizar a divisão no VR)
dados.fillna(0, inplace=True)
dados = dados.query('Patrimonio_Liquido > 0')

In [ ]:
dados = dados.assign(vr_sem_riscos_e_beneficios = dados['Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes']  + dados['Ativo_Direitos_Sem_Aquisicao_Creditos_Inadimplentes'] - dados['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] - dados['Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao'])
dados  = dados.assign(vr_com_riscos_e_beneficios = dados['Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes']  + dados['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes'] - dados['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] - dados['Ativo_Direitos_Aquisicao_Provisao_Reducao'])

dados  = dados.assign(vr = (dados['vr_sem_riscos_e_beneficios'] + dados['vr_com_riscos_e_beneficios']) / dados['Patrimonio_Liquido'])

# mudando VRs negativos para zero
dados['vr'] = dados['vr'].apply(lambda x: x if x >= 0 else 0)

In [ ]:
# filtragem dos dados por ids detectados e ordenação por fundo e data, possibilitando iteração para teste da hipótese 4
dados["Data_Competencia"] = pd.to_datetime(dados["Data_Competencia"], format="%Y-%m-%d")
dados = dados.query(f'ID_Participante in {ids}').reset_index(drop=True).sort_values(by=['ID_Participante', 'Data_Competencia'])

In [ ]:
# colunas utilizadas para cálculo das colunas de teste da hipótese 4
dados['total_a_vencer'] = dados['Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes'] + dados['Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Adimplentes']
dados['total_atrasados'] = dados['Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes'] + dados['Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes']

In [ ]:
# colunas utilizadas para teste da hipótese 4, cujos valores serão atribuídos através de iteração
dados['diff_a_vencer'] = 0
dados['diff_atrasados'] = 0

In [ ]:
#atribuindo valores às colunas de teste da hipótese 4
for i in range(1, len(dados)):
    atual = dados.iloc[i]
    anterior = dados.iloc[i - 1]
    if atual.ID_Participante == anterior.ID_Participante:
        # verificando aumento das parcelas adimplentes de créditos a vencer (atual deveria ser maior que anterior)
        dados.at[i,'diff_a_vencer'] = atual['total_a_vencer'] - anterior['total_a_vencer']
        # verificando diminuição das parcelas inadimplentes de créditos (anterior deveria ser maior que atual)
        dados.at[i,'diff_atrasados'] = anterior['total_atrasados'] - atual['total_atrasados']
    else:
        continue

Dessa forma como foi estabelecido o cálculo na iteração acima, as duas colunas podem ter seus valores mínimos iguais a 0, pois um valor negativo não representaria uma renegociação, enquanto a ocorrência de valores positivos em ambas significa que houve renegociação (caso não haja recompras ou substituições). Isso facilita a visualização dos gráficos nas análises das hipóteses.

In [ ]:
# drop das colunas que não serão mais utilizadas
dados.drop(['total_a_vencer','total_atrasados'], axis=1, inplace=True)

# igualando valores negativos a zero
dados['diff_a_vencer'] = dados['diff_a_vencer'].apply(lambda x: x if x >= 0 else 0)
dados['diff_atrasados'] = dados['diff_atrasados'].apply(lambda x: x if x >= 0 else 0)

Como o valor do VR geralmente não é muito alto, a análise dos gráficos mais abaixo no *notebook* poderia ser prejudicada pela grande diferença de escala (as linhas de valores maiores não permitiriam a visualização clara de alterações no valor do VR). Logo, realizou-se a normalização das colunas numéricas presentes nos dados.

In [ ]:
# normalização dos dados com MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# inicialização do objeto MinMaxScaler
scaler = MinMaxScaler()

# seleção das colunas numéricas dos dados
colunas_numericas = dados.select_dtypes(include='number').columns
colunas_numericas = colunas_numericas.drop(['ID_Participante'])

# aplicação da normalização Min Max às colunas numéricas
dados[colunas_numericas] = scaler.fit_transform(dados[colunas_numericas])


In [ ]:
# lista dos tipos de carteira, ordenada de acordo com a numeração correspondente à coluna "Carteira_Classificação_encoded"
carteiras = [
    'Setor Público',
    'Agronegócio',
    'Cartão',
    'Comercial',
    'Imobiliário',
    'Financeiro',
    'Industrial',
    'Factoring',
    'Multimercado',
    'Serviços',
    'Ações Judiciais'
    ]

### Funções utilizadas

In [ ]:
def plota_grafico_linha_sobre_tempo(dados : pd.DataFrame, colunas : str | list, labels : str | list) -> None:
    fig, ax = plt.subplots(figsize=(15, 5))

    if type(colunas) != str:
        quantidade_colunas = len(colunas)
        for i in range(quantidade_colunas):
            col = colunas[i]
            label = labels[i]
            ax.plot(dados['Data_Competencia'], dados[col], label=label)
        ax.set_title(', '.join(labels) + ' x Tempo')
    else:
        ax.plot(dados['Data_Competencia'], dados[colunas], label=labels)
        ax.set_title(labels + ' x Tempo')

    ax.set_xlabel('Tempo')
    ax.set_ylabel('Valor')
    ax.legend()
    plt.show()

# Hipótese 2
"Fundos com recompras de créditos (3 campos sob o item VII.d) são mais propensos a apresentarem aumento futuro de VR? Qual o nível de recompras antecede um aumento do VR?"

In [ ]:
for i in ids:
    fundo = dados.loc[dados['ID_Participante'] == i]
    plota_grafico_linha_sobre_tempo(fundo, ['vr' ,'Negocios_Recompras_Valor'], ['VR', 'Recompras'])

É possível observar pelos gráficos gerados acima que essa análise é um tanto quanto inconclusiva, porque não há correlação clara entre os valores de recompras e o VR. As oscilações desses valores ocorrem (quando ocorrem) majoritariamente de forma pontual e/ou desconexa, o que, a princípio, impossibilita a validação dessa hipótese com a amostra de dados correspondentes aos resultados do modelo *Isolation Forest*.

# Hipótese 3
"Fundos com substituições de créditos (3 campos sob o item VII.c) são mais propensos a apresentar aumento futuro de VR? Qual o nível de substituições antecede um aumento do VR?"

In [ ]:
for i in ids:
    fundo = dados.loc[dados['ID_Participante'] == i]
    plota_grafico_linha_sobre_tempo(fundo, ['vr' ,'Negocios_Substituicoes_Valor'], ['VR', 'Substituições'])

Os resultados do teste dessa hipótese, explicitados acima, são ainda mais inconclusivos que aqueles da hipótese anterior. Pouquíssimos fundos da amostra apresentam um valor de substituições diferente de zero. Aqueles que o fazem não demonstram correlação entre esse valor e o VR. Logo, a hipótese 3 também não poderia ser validada com os dados da amostra em questão

# Hipótese 4
"FIDCs com renegociações de créditos são mais propensos a apresentar aumento futuro de VR? Qual o nível de renegociações antecede um aumento do VR? As renegociações são identificadas por um aumento de créditos a vencer (campos I.2a.1 e I.2b.1) e diminuição dos valores em atraso (campos I.2a.2.1 e I.2b.2.1 ou V.b e VI.b) no mesmo mês, sem a ocorrência de recompra nem substituição de créditos."

Os gráficos a seguir conterão apenas colunas que dizem respeito ao VR, ao aumento de créditos a vencer ("diff_a_vencer": diferença, no período de um mês, entre parcelas adimplentes de créditos a vencer do ativo do fundo) e à diminuição dos valores em atraso ("diff_atrasados": diferença, no período de um mês, entre parcelas inadimplentes de créditos do ativo do fundo). Isso porque, mesmo a hipótese dependendo da ausência de valores de recompra e substituição, as colunas utilizadas são suficientes para avaliar inicialmente a possibilidade de validação da correlação em questão.

In [ ]:
for i in ids:
    fundo = dados.loc[dados['ID_Participante'] == i]
    plota_grafico_linha_sobre_tempo(fundo, ['vr' ,'diff_a_vencer', 'diff_atrasados'], ['VR', 'Adimplentes a Vencer', 'Inadimplentes'])

Como pode ser observado pelos gráficos acima, as convergências de picos entre o aumento de créditos a vencer ("diff_a_vencer") e a diminuição dos valores em atraso ("diff_atrasados") são quase inexistentes e não apresentam correlação alguma com o VR a princípio. Portanto, não são necessários testes incluindo os valores de recompra e substituição, pois essa análise inicial já demarca que a hipótese não pode ser validada com a amostra utilizada.

# Hipótese 5
"Algum(ns) ramo(s) de atividade econômica do FIDC é(são) mais propenso(s) a registrar maior aumento no VR? A Tabela II é útil para essa análise, porém não traz a divisão de DCs entre com e sem riscos e benefícios."

De acordo com o retorno do modelo *Isolation Forest*, o ramo que mais apresentou dados classificados como anômalos foi o referente à carteira "Comercial", seguido da carteira "Financeiro". Entretanto, esse maior número de ocorrências pode estar intimamente relacionado ao elevado número de registros que possuem a classificação de carteira correspondente a esses dois tipos (classificação de carteira se refere à forma com que foram divididos os dados por tipo de carteira para modelagem, presente nos *notebooks* dos modelos).

É difícil avaliar essa correlação presente na hipótese cinco, porque o modelo *Isolation Forest* não classifica os dados como anômalos baseando-se unicamente em aumentos de VR e tipos de carteira. Verificaremos, portanto, se os gráficos relativos ao valor de risco dos fundos da amostra em análise demonstram um aumento mais frequente em algum tipo de carteira.

In [ ]:
for i in ids:
    numeracao_carteira = dados_tratados[dados_tratados['ID_Participante'] == i]['Carteira_Classificação_encoded'].unique()[0]
    tipo_carteira = carteiras[numeracao_carteira]
    print(f'Tipo da carteira do fundo: {tipo_carteira}')
    fundo = dados.loc[dados['ID_Participante'] == i]
    plota_grafico_linha_sobre_tempo(fundo, 'vr','VR')

A partir da visualização dos gráficos acima, é possível concluir que, em média, os fundos da amostra classificados nas carteiras de Agronegócio, Financeiro e Multimercado apresentam um volume maior de dados com aumentos no valor do VR (por conter apenas um registro na amostra, a carteira de Factoring não foi selecionada). Porém, não é viável determinar qual tipo de carteira possui mais tendências de aumento de VR de acordo com essa análise ou com qualquer resultado retornado pelo modelo *Isolation Forest*. Por isso, essa hipótese não pode ser validada ou mesmo considerada para teste com os dados da amostra.